In [1]:
!git clone https://github.com/microsoft/GenerativeImage2Text.git
%cd GenerativeImage2Text
!pip install -r requirements.txt
!python setup.py build develop

Cloning into 'GenerativeImage2Text'...
remote: Enumerating objects: 160, done.
remote: Counting objects: 100% (63/63), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 160 (delta 44), reused 33 (delta 26), pack-reused 97
Receiving objects: 100% (160/160), 501.71 KiB | 6.19 MiB/s, done.
Resolving deltas: 100% (63/63), done.
/content/GenerativeImage2Text


In [ ]:
!python -m generativeimage2text.train -p "{'type': 'speed_test_forward_backward'}"

2024-02-25 14:33:33,404.404 14171:train.py:309   <module>(): param:
{'type': 'speed_test_forward_backward'}
2024-02-25 14:33:35,410.410 14171:train.py:274 speed_test_forward_backward(): SelectTransform(ts=[Compose(
    ImageTransform2Dict(image_transform=Compose(
    RandomResizedCrop(size=(160, 160), scale=(0.8, 1.0), ratio=(1.0, 1.0), interpolation=bicubic, antialias=warn)
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
))
), Compose(
    ImageTransform2Dict(image_transform=Compose(
    RandomResizedCrop(size=(176, 176), scale=(0.8, 1.0), ratio=(1.0, 1.0), interpolation=bicubic, antialias=warn)
    ToTensor()
    Normalize(mean=(0.48145466, 0.4578275, 0.40821073), std=(0.26862954, 0.26130258, 0.27577711))
))
), Compose(
    ImageTransform2Dict(image_transform=Compose(
    RandomResizedCrop(size=(192, 192), scale=(0.8, 1.0), ratio=(1.0, 1.0), interpolation=bicubic, antialias=warn)
    ToTensor()
    Normalize(mean=(0.481

In [4]:
# single image, captioning
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': 'aux_data/images/1.jpg', \
      'model_name': 'GIT_BASE', \
      'prefix': '', \
}"
# single image, question answering
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': 'aux_data/images/1.jpg', \
      'model_name': 'GIT_BASE_VQAv2', \
      'prefix': 'what is it?', \
}"
# multiple images, captioning
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': ['aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg'], \
      'model_name': 'GIT_BASE_VATEX', \
      'prefix': '', \
}"
# multiple images, question answering
!AZFUSE_TSV_USE_FUSE=1 python -m generativeimage2text.inference -p "{'type': 'test_git_inference_single_image', \
      'image_path': ['aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg', 'aux_data/images/1.jpg'], \
      'model_name': 'GIT_BASE_MSRVTT_QA', \
      'prefix': 'what is it?', \
}"

2024-02-25 12:56:52,618.618 7181:inference.py:318   <module>(): param:
{'image_path': 'aux_data/images/1.jpg',
 'model_name': 'GIT_BASE',
 'prefix': '',
 'type': 'test_git_inference_single_image'}
2024-02-25 12:56:57,744.744 7181:cloud_storage.py:1150 az_download_once(): if sync, no need to save to temp first
2024-02-25 12:56:57,745.745 7181:common.py:324    cmd_run(): start to cmd run: azcopy cp https://publicgit.blob.core.windows.net/data/output/GIT_BASE/snapshot/model.pt /tmp/publicgit/output/GIT_BASE/snapshot/model.pt
2024-02-25 12:56:57,746.746 7181:cloud_storage.py:1211 az_download_once(): ['azcopy',
 'cp',
 'https://publicgit.blob.core.windows.net/data/output/GIT_BASE/snapshot/model.pt',
 '/tmp/publicgit/output/GIT_BASE/snapshot/model.pt']
2024-02-25 12:56:57,746.746 7181:common.py:234 limited_retry_agent(): fails with 
[Errno 2] No such file or directory: 'azcopy': tried 1/5-th time
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/azfuse/common